In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [18]:
df = pd.read_csv('https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/002/492/original/ola_driver_scaler.csv', index_col=0)

In [19]:
df.head(10)

,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,12/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
5,12/01/19,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
6,01/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
7,02/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
8,03/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,350000,1
9,04/01/20,4,43.0,0.0,C13,2,65603,12/07/19,27/04/20,2,2,0,1


In [20]:
df.describe()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating
count,19104.000000,19043.000000,19052.000000,19104.000000,19104.000000,19104.000000,19104.000000,1.910400e+04,19104.000000
mean,1415.591133,34.668435,0.418749,1.021671,65652.025126,1.690536,2.252670,5.716621e+05,2.008899
std,810.705321,6.257912,0.493367,0.800167,30914.515344,0.836984,1.026512,1.128312e+06,1.009832
min,1.000000,21.000000,0.000000,0.000000,10747.000000,1.000000,1.000000,-6.000000e+06,1.000000
25%,710.000000,30.000000,0.000000,0.000000,42383.000000,1.000000,1.000000,0.000000e+00,1.000000
50%,1417.000000,34.000000,0.000000,1.000000,60087.000000,1.000000,2.000000,2.500000e+05,2.000000
75%,2137.000000,39.000000,1.000000,2.000000,83969.000000,2.000000,3.000000,6.997000e+05,3.000000
max,2788.000000,58.000000,1.000000,2.000000,188418.000000,5.000000,5.000000,3.374772e+07,4.000000


In [21]:
df.describe(include=[object])

,MMM-YY,City,Dateofjoining,LastWorkingDate
count,19104,19104,19104,1616
unique,24,29,869,493
top,01/01/19,C20,23/07/15,29/07/20
freq,1022,1008,192,70


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19104 entries, 0 to 19103
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MMM-YY                19104 non-null  object 
 1   Driver_ID             19104 non-null  int64  
 2   Age                   19043 non-null  float64
 3   Gender                19052 non-null  float64
 4   City                  19104 non-null  object 
 5   Education_Level       19104 non-null  int64  
 6   Income                19104 non-null  int64  
 7   Dateofjoining         19104 non-null  object 
 8   LastWorkingDate       1616 non-null   object 
 9   Joining Designation   19104 non-null  int64  
 10  Grade                 19104 non-null  int64  
 11  Total Business Value  19104 non-null  int64  
 12  Quarterly Rating      19104 non-null  int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 2.0+ MB


In [23]:
df.isna().sum()

MMM-YY                      0
Driver_ID                   0
Age                        61
Gender                     52
City                        0
Education_Level             0
Income                      0
Dateofjoining               0
LastWorkingDate         17488
Joining Designation         0
Grade                       0
Total Business Value        0
Quarterly Rating            0
dtype: int64

In [26]:
df['Dateofjoining'] = pd.to_datetime(df['Dateofjoining'])
df['LastWorkingDate'] = pd.to_datetime(df['LastWorkingDate'])

# Perform the subtraction
df['Difference'] = df['LastWorkingDate'] - df['Dateofjoining'] 



In [28]:
df.nunique()

MMM-YY                     24
Driver_ID                2381
Age                        36
Gender                      2
City                       29
Education_Level             3
Income                   2383
Dateofjoining             869
LastWorkingDate           493
Joining Designation         5
Grade                       5
Total Business Value    10181
Quarterly Rating            4
Difference                680
dtype: int64

In [29]:
df.drop('MMM-YY', axis=1, inplace=True)

In [30]:
df

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating,Difference
0,1,28.0,0.0,C23,2,57387,2018-12-24,NaT,1,1,2381060,2,NaT
1,1,28.0,0.0,C23,2,57387,2018-12-24,NaT,1,1,-665480,2,NaT
2,1,28.0,0.0,C23,2,57387,2018-12-24,2019-03-11,1,1,0,2,77 days
3,2,31.0,0.0,C7,2,67016,2020-11-06,NaT,2,2,0,1,NaT
4,2,31.0,0.0,C7,2,67016,2020-11-06,NaT,2,2,0,1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2788,30.0,0.0,C27,2,70254,2020-06-08,NaT,2,2,740280,3,NaT
19100,2788,30.0,0.0,C27,2,70254,2020-06-08,NaT,2,2,448370,3,NaT
19101,2788,30.0,0.0,C27,2,70254,2020-06-08,NaT,2,2,0,2,NaT
19102,2788,30.0,0.0,C27,2,70254,2020-06-08,NaT,2,2,200420,2,NaT


In [15]:
df['LastWorkingDate'].fillna('', inplace=True)

In [32]:
df.groupby(['Driver_ID']).max().reset_index()

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating,Difference
0,1,28.0,0.0,C23,2,57387,2018-12-24,2019-03-11,1,1,2381060,2,77 days
1,2,31.0,0.0,C7,2,67016,2020-11-06,NaT,2,2,0,1,NaT
2,4,43.0,0.0,C13,2,65603,2019-12-07,2020-04-27,2,2,350000,1,142 days
3,5,29.0,0.0,C9,0,46368,2019-01-09,2019-03-07,1,1,120360,1,57 days
4,6,31.0,1.0,C11,1,78728,2020-07-31,NaT,3,3,1265000,2,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,34.0,0.0,C24,0,82815,2015-10-15,NaT,2,3,4495040,4,NaT
2377,2785,34.0,1.0,C9,0,12105,2020-08-28,2020-10-28,1,1,0,1,61 days
2378,2786,45.0,0.0,C19,0,35370,2018-07-31,2019-09-22,2,2,970380,2,418 days
2379,2787,28.0,1.0,C20,2,69498,2018-07-21,2019-06-20,1,1,408090,2,334 days
